In [10]:
# %pip install pandas numpy matplotlib scikit-learn statsmodels plotly openpyxl xlrd

# Alcohol (per capita, 15+) vs Happiness
- Inputs:
    --alcohol   : path to EE6F72A_ALL_LATEST.csv
    --happiness : path to WHR25_Data_Figure_2.1.xlsx
- Outputs (in ./out):
    scatter_fits_<year>.png
    k_silhouette_<year>.png
    clusters_scatter_<year>.png
    clusters_map_<year>.html     (interactive choropleth)
    cluster_summary_<year>.csv
    merged_<year>.csv

In [11]:
import argparse
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.nonparametric.smoothers_lowess import lowess
import plotly.express as px

In [12]:
def load_alcohol(alcohol_path: str) -> pd.DataFrame:
    """Load alcohol dataset and normalize to columns: country, year, alcohol_per_capita_litres."""
    df = pd.read_csv(alcohol_path)

    # Known column names from EE6F72A_ALL_LATEST.csv
    rename_map = {
        "GEO_NAME_SHORT": "country",
        "DIM_TIME": "year",
        "RATE_PER_CAPITA_N": "alcohol_per_capita_litres",
    }
    for k, v in rename_map.items():
        if k in df.columns:
            df = df.rename(columns={k: v})

    # If per-capita column not present but total & population exist, compute it.
    if "alcohol_per_capita_litres" not in df.columns:
        if {"total_alcohol_litres", "adult_population_15p"}.issubset(df.columns):
            df["alcohol_per_capita_litres"] = (
                df["total_alcohol_litres"] / df["adult_population_15p"]
            )
        elif "alcohol" in df.columns:
            df = df.rename(columns={"alcohol": "alcohol_per_capita_litres"})
        else:
            raise ValueError(
                "Could not find per-capita alcohol column. "
                "Expected RATE_PER_CAPITA_N or (total_alcohol_litres & adult_population_15p)."
            )

    # Keep only essential columns
    keep_cols = ["country", "year", "alcohol_per_capita_litres"]
    missing = [c for c in keep_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Alcohol CSV missing columns: {missing}")

    df = df[keep_cols].copy()
    df = df.dropna()
    df["year"] = df["year"].astype(int)
    return df


def load_happiness(happiness_path: str) -> pd.DataFrame:
    """Load happiness dataset from WHR25_Data_Figure_2.1.xlsx (sheet 'Data for Figure 2.1 (2011–2024)')."""
    # The workbook may contain multiple sheets; we need the named one
    sheet_name = "Data for Figure 2.1 (2011–2024)"
    wb = pd.read_excel(happiness_path, sheet_name=sheet_name)

    rename_map = {
        "Country name": "country",
        "Year": "year",
        "Ladder score": "happiness",
    }
    for k, v in rename_map.items():
        if k in wb.columns:
            wb = wb.rename(columns={k: v})

    keep_cols = ["country", "year", "happiness"]
    missing = [c for c in keep_cols if c not in wb.columns]
    if missing:
        # Fallback: try to detect similar columns
        raise ValueError(f"Happiness sheet missing columns: {missing}. "
                         f"Check the sheet name or column headers.")

    wb = wb[keep_cols].copy()
    wb = wb.dropna()
    wb["year"] = wb["year"].astype(int)
    return wb


def latest_common_year(alc: pd.DataFrame, hap: pd.DataFrame, prefer=None) -> int:
    commons = sorted(set(alc["year"]).intersection(set(hap["year"])))
    if not commons:
        raise ValueError("No common years between alcohol and happiness datasets.")
    if prefer and prefer in commons:
        return prefer
    return commons[-1]


def merge_year(alc: pd.DataFrame, hap: pd.DataFrame, year: int) -> pd.DataFrame:
    a = alc.loc[alc["year"] == year].copy()
    h = hap.loc[hap["year"] == year].copy()

    # Harmonize country names if needed (light touch)
    # (You can extend with a mapping dict for tricky cases.)
    merged = pd.merge(a, h, on=["country", "year"], how="inner")
    merged = merged.dropna()
    merged = merged[
        (merged["alcohol_per_capita_litres"] >= 0) & (merged["happiness"].between(0, 10))
    ]
    return merged


def scatter_with_fits(df: pd.DataFrame, year: int, outdir: Path):
    x = df["alcohol_per_capita_litres"].values.reshape(-1, 1)
    y = df["happiness"].values

    # Correlations
    pearson = df["alcohol_per_capita_litres"].corr(df["happiness"], method="pearson")
    spearman = df["alcohol_per_capita_litres"].corr(df["happiness"], method="spearman")

    # OLS
    ols = LinearRegression().fit(x, y)
    x_line = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
    y_ols = ols.predict(x_line)

    # Polynomial (deg=2)
    poly = make_pipeline(PolynomialFeatures(2, include_bias=False), LinearRegression())
    poly.fit(x, y)
    y_poly = poly.predict(x_line)

    # LOWESS smoothing
    lo = lowess(y, x.flatten(), frac=0.6, return_sorted=True)

    plt.figure(figsize=(8, 5))
    plt.scatter(x, y, alpha=0.6)
    plt.plot(x_line, y_ols, linewidth=2, label="OLS")
    plt.plot(x_line, y_poly, linewidth=2, linestyle="--", label="Polynomial (deg=2)")
    plt.plot(lo[:, 0], lo[:, 1], linewidth=2, linestyle=":", label="LOWESS")
    plt.xlabel("Alcohol per capita (litres of pure alcohol, 15+)")
    plt.ylabel("Happiness (0–10)")
    plt.title(f"Alcohol vs Happiness • {year}\nPearson={pearson:.2f}, Spearman={spearman:.2f}")
    plt.legend()
    plt.tight_layout()
    outpath = outdir / f"scatter_fits_{year}.png"
    plt.savefig(outpath, dpi=160)
    plt.close()
    print(f"[saved] {outpath}")
    return pearson, spearman


def kmeans_clustering(df: pd.DataFrame, year: int, outdir: Path):
    X = df[["alcohol_per_capita_litres", "happiness"]].values
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    # Choose best k by silhouette (2..6)
    best_k, best_score = None, -1
    ks, ss = [], []
    for k in range(2, 7):
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
        labels = km.fit_predict(Xs)
        score = silhouette_score(Xs, labels)
        ks.append(k)
        ss.append(score)
        if score > best_score:
            best_k, best_score = k, score

    # Silhouette plot
    plt.figure(figsize=(6, 4))
    plt.plot(ks, ss, marker="o")
    plt.title(f"Silhouette score by k • {year}")
    plt.xlabel("k")
    plt.ylabel("Silhouette score")
    plt.tight_layout()
    outpath_s = outdir / f"k_silhouette_{year}.png"
    plt.savefig(outpath_s, dpi=160)
    plt.close()
    print(f"[saved] {outpath_s}")

    # Fit best k
    km = KMeans(n_clusters=best_k, n_init=10, random_state=42)
    labels = km.fit_predict(Xs)
    centers = scaler.inverse_transform(km.cluster_centers_)

    # Scatter with clusters
    plt.figure(figsize=(8, 5))
    plt.scatter(df["alcohol_per_capita_litres"], df["happiness"], c=labels, alpha=0.75)
    plt.scatter(centers[:, 0], centers[:, 1], marker="x", s=120)
    plt.xlabel("Alcohol per capita (L, 15+)")
    plt.ylabel("Happiness (0–10)")
    plt.title(f"KMeans clusters (k={best_k}) • {year}")
    plt.tight_layout()
    outpath_c = outdir / f"clusters_scatter_{year}.png"
    plt.savefig(outpath_c, dpi=160)
    plt.close()
    print(f"[saved] {outpath_c}")

    df_out = df.copy()
    df_out["cluster"] = labels
    return df_out, best_k, best_score


def choropleth_map(df_out: pd.DataFrame, year: int, outdir: Path):
    """Interactive map with Plotly (no internet required)."""
    fig = px.choropleth(
        df_out,
        locations="country",
        locationmode="country names",
        color="cluster",
        hover_name="country",
        hover_data=["alcohol_per_capita_litres", "happiness"],
        title=f"Alcohol vs Happiness • KMeans clusters (k={df_out['cluster'].nunique()}) • {year}",
        color_continuous_scale=['#1f77b4', '#ff7f0e']
    )
    outpath = outdir / f"clusters_map_{year}.html"
    fig.write_html(str(outpath), include_plotlyjs="cdn")
    print(f"[saved] {outpath}")

In [13]:
outdir = Path("out")
outdir.mkdir(exist_ok=True, parents=True)

alc = load_alcohol('data/EE6F72A_ALL_LATEST.csv')
hap = load_happiness('data/WHR25_Data_Figure_2.1.xlsx')

In [14]:
alc.head()

,country,year,alcohol_per_capita_litres
0,Cuba,2003,5.885221
1,Cyprus,2003,7.142592
2,Least developed countries,2003,2.029877
3,Sub-Saharan Africa,2003,3.860405
4,Czechia,2003,14.557763


In [15]:
hap.head()

,country,year,happiness
0,Finland,2024,7.736
1,Afghanistan,2023,1.721
2,Afghanistan,2022,1.859
3,Afghanistan,2021,2.404
4,Afghanistan,2020,2.523


In [16]:
year = latest_common_year(alc, hap)
print(f"year: {year}")

merged = merge_year(alc, hap, year)
merged.to_csv(outdir / f"merged_{year}.csv", index=False)
print(merged.head())
print(f"[info] merged rows: {len(merged)} (saved to out/merged_{year}.csv)")

year: 2022
        country  year  alcohol_per_capita_litres  happiness
0        Serbia  2022                   9.031430      6.144
1  Sierra Leone  2022                   0.333927      3.138
2     Singapore  2022                   1.957899      6.587
3      Slovakia  2022                  10.971754      6.469
4      Viet Nam  2022                  10.715408      5.763
[info] merged rows: 122 (saved to out/merged_2022.csv)


In [17]:
merged.head()

,country,year,alcohol_per_capita_litres,happiness
0,Serbia,2022,9.031430,6.144
1,Sierra Leone,2022,0.333927,3.138
2,Singapore,2022,1.957899,6.587
3,Slovakia,2022,10.971754,6.469
4,Viet Nam,2022,10.715408,5.763


In [18]:
pearson, spearman = scatter_with_fits(merged, year, outdir)
df_out, best_k, best_score = kmeans_clustering(merged, year, outdir)

# Save cluster summary & map
summary = df_out.groupby("cluster").agg(
    countries=("country", list),
    n=("country", "count"),
    mean_alcohol=("alcohol_per_capita_litres", "mean"),
    mean_happiness=("happiness", "mean"),
).reset_index()
summary_path = outdir / f"cluster_summary_{year}.csv"
summary.to_csv(summary_path, index=False)
print(f"[saved] {summary_path}")
choropleth_map(df_out, year, outdir)

print("\n=== Summary ===")
print(f"Year: {year}")
print(f"Pearson r:  {pearson:.3f}")
print(f"Spearman ρ: {spearman:.3f}")
print(f"Best k:     {best_k} (silhouette={best_score:.3f})")
print("Outputs saved in ./out")

[saved] out\scatter_fits_2022.png
[saved] out\k_silhouette_2022.png
[saved] out\clusters_scatter_2022.png
[saved] out\cluster_summary_2022.csv
[saved] out\clusters_map_2022.html

=== Summary ===
Year: 2022
Pearson r:  0.570
Spearman ρ: 0.601
Best k:     2 (silhouette=0.451)
Outputs saved in ./out


C:\Users\UserK\AppData\Local\Temp\ipykernel_908\2387444828.py:184: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



# 📊 주요 결과

## 상관계수
- Pearson r ≈ 0.57
- Spearman ρ ≈ 0.60

> 음주량과 행복지수 간에 중간 정도의 양의 상관관계가 존재

## 비선형 분석

- 선형 회귀(OLS)도 양의 기울기
- 2차 다항 회귀, LOWESS 모두 비슷한 패턴

> "적당히 마시는 국가일수록 행복지수 높음" 경향
> 다만 극단적으로 높은 음주량(15L 이상)에서는 행복지수가 더는 올라가지 않음

## 클러스터링 (KMeans, 최적 k=2, 실루엣≈0.45)

- 군집 1: 낮은 음주량(0~5L) + 낮은 행복지수(평균 ~4.7) → 주로 저개발국
- 군집 2: 중간높은 음주량(612L) + 높은 행복지수(평균 ~6.3) → 유럽·북미 선진국